In [ ]:
%matplotlib notebook
import scipy.optimize as opti
import numpy as np
import matplotlib as mpl
from matplotlib import pyplot as plt
from matplotlib.widgets import CheckButtons, SpanSelector
import ipywidgets as widgets
from ipywidgets import interact #, interactive, fixed, interact_manual
import pandas as pd
from scipy.optimize import minimize
import price_utils as putils
import data_utils as dutils

In [ ]:
fileName = "dataid1103.csv"
numOfDays = 1
sinceDayId = 1

firstTime = -3
def onInteractEvent(fN, nD, sD):
    global fileName, numOfDays, sinceDayId, firstTime
    
    firstTime += 1
    if firstTime > 0:
        fileName, numOfDays, sinceDayId = fN, nD, sD
        
        x, grid, cons, prod = dutils.init_data(fileName, numOfDays, sinceDayId)
        
        optimize(x, grid, cons, prod)

def onSelectHouse(x):
    global numOfDays, sinceDayId
    onInteractEvent(x, numOfDays, sinceDayId)

def onNumOfDays(x):
    global fileName, sinceDayId
    onInteractEvent(fileName, x, sinceDayId)
    
def onSinceDay(x):
    global fileName, numOfDays
    onInteractEvent(fileName, numOfDays, x)

interact(onSelectHouse, x=dutils.get_houses_csvs());
interact(onSinceDay, x=dutils.get_dict_day_id(fileName));
interact(onNumOfDays, x=widgets.IntSlider(min=1,max=30,step=1,value=1));
plt.plot(figsize=(10,4))

In [ ]:
def costFuncOnMinE(X):
    batt_diff = np.insert(np.diff(X), 0, 0)
    return sum(np.array(CONS + batt_diff - PROD).clip(0,100))

def newGridEvolution(X):
    batt_diff = np.insert(np.diff(X), 0, 0)
    return np.array(CONS + batt_diff - PROD).clip(0,100)

def costFuncOnMinPrice(X):
    batt_diff = np.insert(np.diff(X), 0, 0)
    return sum(np.array(CONS + batt_diff - PROD).clip(0,100) * PRICE)

def costEvolutionProduction(X):
    D = np.insert(np.diff(X), 0, 0).clip(0, 100)
    return PROD - (PROD - (CONS + D)).clip(0,100)
#     unused_prod = np.array(CONS + D - PROD).clip(-100, 0)
#     return PROD + unused_prod
    

In [ ]:
X_DATA, PRICE, GRID, CONS, PROD, RESULT = [], [], [], [], [], {}
prod_usage = None
def optimize(x, grid, cons, prod):
    global X_DATA, PRICE, GRID, CONS, PROD, RESULT
    X_DATA, GRID, CONS, PROD, PRICE = x, grid, cons, prod, putils.get_prices_per_hour(x)
    b = [(0,0)] + [(0,10)] * (len(CONS) - 1)
    
    RESULT = opti.minimize(costFuncOnMinE, #costFuncOnMinE costFuncOnMinPrice
                           np.repeat(0,len(CONS)), 
                           bounds=b, 
                           method='L-BFGS-B',
                           options={'disp': False, 'maxiter': 15000, 'maxfun': 3000000}) # L-BFGS-B SLSQP
    
    new_grid = newGridEvolution(RESULT.x)
    prod_usage = costEvolutionProduction(RESULT.x)
    showGraph(x=x, 
              grid=grid, 
              cons=cons, 
              prod=prod,
              batt_charge=RESULT.x, 
              new_grid=new_grid,
              prod_usage=prod_usage)
    report(x=x, price=PRICE, new_grid=new_grid, grid=grid, cons=cons, prod=prod, prod_usage=prod_usage)
    

In [ ]:
def showGraph(x=[], grid=[], cons=[], prod=[], batt_charge=[], new_grid=[], prod_usage=[]):
    plt.clf()
    plt.ylim(-3, 10)
    plt.plot(x, prod, c='g', label='Load Prod')
    plt.plot(x, grid, c='b', label='Load Grid')
    plt.plot(x, cons, c='r', ls='dashed', label='Load Cons')
    plt.plot(x, new_grid, c='m', label='Load New Grid')
#     plt.plot(x, batt_charge, c='y', label='Battery Charge')
    plt.plot(x, prod_usage, c='k', ls='dotted', label="Production Usage")
    
    plt.legend()
    plt.tight_layout()
    plt.show()

In [ ]:
def report(x=[], price=[], new_grid=[], grid=[], cons=[], prod=[], prod_usage=[]):
    price_grid = putils.get_cost_per_hour(price, new_grid)
    print("Price ", price_grid.sum())
    print("New Grid ", new_grid.sum())
    print("E from Phtovoltaic ", prod_usage.sum())
    print("-check- Grid ", grid.clip(0, 100).sum())
    print("-check- Cons ", cons.sum())
    print("-check- Prod ", prod.sum())
    print("------------------")
    print("RESULT.fu ", RESULT.fun)
    print("RESULT.success ", RESULT.success)
    print("RESULT.message ", RESULT.message)
    print("RESULT.nfev ", RESULT.nfev)
    print("RESULT.nit ", RESULT.nit)
